In [3]:
import nltk
import re
from nltk.corpus import inaugural, stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import svd
nltk.data.path = []
nltk.data.path.append('/Users/jiaenchang/nltk_data')
print(nltk.data.path)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('inaugural')

['/Users/jiaenchang/nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jiaenchang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jiaenchang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package inaugural to
[nltk_data]     /Users/jiaenchang/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!


True

(a) Write the load_data function to load inaugural corpus and pre-process the text: remove non-letters, convert into the lower case, and remove the stop words.

In [ ]:
def load_data():
    ####### TO DO #######

    stop_words = set(stopwords.words('english'))
    list_words = []
    
    # Loop through all the speeches in the inaugural corpus
    for fileid in inaugural.fileids():
        # Tokenize the speech
        words = word_tokenize(inaugural.raw(fileid))
        
        # Remove non-letters, convert to lowercase, and remove stopwords
        processed_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
        
        list_words.append(processed_words)
    
    return list_words

    ######################
    
inaugural_corpus = load_data()
print(inaugural_corpus[:3])


[['senate', 'house', 'representatives', 'among', 'vicissitudes', 'incident', 'life', 'event', 'could', 'filled', 'greater', 'anxieties', 'notification', 'transmitted', 'order', 'received', 'day', 'present', 'month', 'one', 'hand', 'summoned', 'country', 'whose', 'voice', 'never', 'hear', 'veneration', 'love', 'retreat', 'chosen', 'fondest', 'predilection', 'flattering', 'hopes', 'immutable', 'decision', 'asylum', 'declining', 'years', 'retreat', 'rendered', 'every', 'day', 'necessary', 'well', 'dear', 'addition', 'habit', 'inclination', 'frequent', 'interruptions', 'health', 'gradual', 'waste', 'committed', 'time', 'hand', 'magnitude', 'difficulty', 'trust', 'voice', 'country', 'called', 'sufficient', 'awaken', 'wisest', 'experienced', 'citizens', 'distrustful', 'scrutiny', 'qualifications', 'could', 'overwhelm', 'despondence', 'one', 'inheriting', 'inferior', 'endowments', 'nature', 'unpracticed', 'duties', 'civil', 'administration', 'ought', 'peculiarly', 'conscious', 'deficiencies',

(b)Write the  distinct_words  function to return the generated from the corpus. Function should return words where words[i] is the ith sorted distinct word from the pre-processed corpus. 

In [ ]:
def distinct_words(corpus):
    ######## TO DO ########

    # Flatten the corpus into a list of words
    all_words = [word for doc in corpus for word in doc]
    
    # Get distinct words and sort them
    corpus_words = sorted(set(all_words))
    
    # Number of distinct words
    num_corpus_words = len(corpus_words)
    
    return corpus_words, num_corpus_words

    #######################


words, num_words = distinct_words(inaugural_corpus)
print(words[:10])

['abandon', 'abandoned', 'abandonment', 'abate', 'abdicated', 'abeyance', 'abhorring', 'abide', 'abiding', 'abilities']


(c) Write   the   function  compute_co_occurrence_matrix to return co-occurrence matrix M and a dictionary word2Ind (which can be used to determine the index of a given word in words).

In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=10):
    ######## TO DO ########

    # Build a dictionary to map words to indices
    word2Ind = {word: idx for idx, word in enumerate(words)}
    vocab_size = len(words)
    
    # Initialize the co-occurrence matrix with zeros
    M = np.zeros((vocab_size, vocab_size), dtype=int)
    
    # Loop through each document in the corpus
    for doc in corpus:
        # For each word in the document
        for i, target_word in enumerate(doc):
            target_idx = word2Ind[target_word]
            
            # Get the window around the target word
            start = max(i - window_size, 0)
            end = min(i + window_size + 1, len(doc))
            
            # Count co-occurrences
            for j in range(start, end):
                if i != j:  # Avoid co-occurrence with itself
                    context_word = doc[j]
                    context_idx = word2Ind[context_word]
                    M[target_idx][context_idx] += 1
    
    return M, word2Ind

    #######################


M, word2Ind = compute_co_occurrence_matrix(inaugural_corpus)

(d)Write the function  perform_svd  taking co-occurrence matrix M and integer k, and return there matrices U,S,V where U represents the k-dimensional word embedding.

In [ ]:
def perform_svd(matrix, k):

    ######## TO DO ########

    # Perform full SVD
    U, S, Vh = svd(matrix)
    
    # Keep only the top k components
    U_reduced = U[:, :k]
    S_reduced = np.diag(S[:k])
    Vh_reduced = Vh[:k, :]
    
    return U_reduced, S_reduced, Vh_reduced

    #######################



U, S, Vh = perform_svd(M, 300)

(e) Write   the   function  train_word2vec_model  which  uses  the gensim’s word2vec model to train a corpus to generate a word embedding. 

In [ ]:
def train_word2vec_model(corpus,
                         num_features=300,
                         min_word_count=40,
                         num_workers=4,
                         context=10,
                         downsampling=1e-3):
    
    ######## TO DO ########

    model = Word2Vec(corpus, vector_size=num_features, window=context, min_count=min_word_count, workers=num_workers, sg=0)
    return model

    #######################    


word2vec_model = train_word2vec_model(inaugural_corpus)

(f) Finally, use the most_similar function from gensim’s word2vec model to list the words which are most similar to ‘government. Write a function most_similar that also lists the words most similar to a given word using the embedding obtained from co-occurrence matrix.

In [ ]:
def most_similar(word, words, word2Ind, U, top_n=10):

    if word not in word2Ind:
        return []
    
    word_idx = word2Ind[word]
    word_vector = U[word_idx]
    
    # Compute cosine similarity between the target word and all other words
    similarities = cosine_similarity([word_vector], U)
    similarities = similarities.flatten()
    
    # Get the indices of the top_n most similar words
    top_indices = similarities.argsort()[-top_n-1:-1][::-1]
    
    # Create a list of tuples (word, similarity_score)
    similar_words = [(words[idx], similarities[idx]) for idx in top_indices]
    
    return similar_words


In [10]:
most_similar("government", words, word2Ind, U)

[('curb', 0.08271793889213876),
 ('faire', 0.07815378336422665),
 ('metropolis', 0.07622967558408168),
 ('arose', 0.07309154009507515),
 ('unnatural', 0.06792193381135365),
 ('writers', 0.0674900289870798),
 ('insurmountable', 0.06521574162041055),
 ('incompatible', 0.06420815195327755),
 ('unconstitutional', 0.06330624641716949),
 ('laissez', 0.06248785352198268)]

In [11]:
word2vec_model.wv.most_similar("government")

[('general', 0.9996548891067505),
 ('state', 0.9996531009674072),
 ('executive', 0.9996502995491028),
 ('rights', 0.999640166759491),
 ('respect', 0.9996368288993835),
 ('legislation', 0.9996331930160522),
 ('powers', 0.9996307492256165),
 ('interests', 0.9996298551559448),
 ('law', 0.9996248483657837),
 ('upon', 0.9996204972267151)]